# THIES Binary Processor

THIES DL16 exports two types of data in two different directories: 
* `ARCH_AV1` for avarage values of a 10 min span. Consists of 19 parameters of DataType Float.
* `ARCH_EX1` for both min and max values measured within the 10 min span. Consists of 32 parameters of DataType FloatExtrem.

There is a file for each day measured. File names are in `YYYYMMDD.BIN` format.

In [ ]:
import numpy as np
from bitarray import bitarray
import pandas as pd
import configparser
import struct

In [6]:
# Size of row (bytes)
AV_ROW_SIZE = 99
EX_ROW_SIZE = 292
# Size (n of parameters)
AV_DESC_SIZE = 19
EX_DESC_SIZE = 32
# IGNORE first 4 bytes of each row (timestamp?) PENDIENTE !!!
OFFSET = 4 
STATUS_CHAR = {0b00000000 : 0,  # Status OK
              0b10000000 : '-', # Sensor is deactivated in the sensor configuration
              0b01000000 : '-', # Datalogger is in maintenance mode
              0b00100000 : '%', # Timeout (for ex. digitalization takes too long)
              0b00010000 : '!', # Value is out of valid range
              0b00001000 : '@', # Difference between 2 consecutive values is too far
              0b00000100 : '#', # Filling level of the averaging buffer is too low
              0b00000010 : '?', # Error depending on measurement type (for ex. ADC overflow)
              0b00000001 : '?'} # Error depending on measurement type (for ex. cable break)

In [8]:
path_bin_av = './BINFILES/ARCH_AV1/20240531.BIN'
path_bin_ex = './BINFILES/ARCH_EX1/20240531.BIN'
path_ini_av = './BINFILES/ARCH_AV1/DESCFILE.INI'
path_ini_ex = './BINFILES/ARCH_EX1/DESCFILE.INI'

## Class THIESData


Por hacer:

* Función para guardar en .txt u otros. Ver el caso especial del formato de datos EX (mezclar dataDF con datesDF en .txt)
* Opción de procesar múltiples archivos .BIN para obtener un solo .txt


In [170]:
class THIESData:
    # Bytes per row
    BPR = {'av': 99, 'ex': 292}
    # Parameters per row
    PPR = {'av': 19, 'ex': 32}
    # Bytes per parameter
    BPP = {'av': 5, 'ex': 9}
    # Timestamp Offset
    OFFSET = 4 

    def __init__(self, binfile_path: str, inifile_path: str, datatype: str) -> None:
        d = datatype.lower().strip() 
        if d not in ['av', 'ex']:
            raise ValueError("Invalid datatype. Expected 'av' (average values) or 'ex' (minmax values).")
        
        self._bpr = THIESData.BPR[d]    
        self._bpp = THIESData.BPP[d]    
        self._datatype = datatype
        self._binfile = THIESData._read_binfile(binfile_path)
        self.descfile = THIESData._read_descfile(inifile_path)
        self.nparameters = len(self.descfile)
        self.nbytes = len(self._binfile)
        self.nrows = int(self.nbytes / self._bpr)
        self.statusDF = None
        self.dataDF = None
        self.datesDF = None  

        self._make_dataframe()

    @staticmethod
    def _bytes2datetime(b: bytes, only_time=False):
        '''
        Input: bytes (size 4)
        Output: str (YYYY/MM/DD hh:mm:ss)
        '''
        bits = bitarray()
        bits.frombytes(b[::-1]) # Invert 4 bytes
        hr = int(bits[15:20].to01(),2)
        min = int(bits[20:26].to01(),2)
        sec = int(bits[26:].to01(),2)
        time = f'{str(hr).zfill(2)}:{str(min).zfill(2)}'
        if only_time:
            return time
        yr = int(bits[0:6].to01(),2)
        mon = int(bits[6:10].to01(),2)
        day = int(bits[10:15].to01(),2)
        date = f'20{yr}/{str(mon).zfill(2)}/{str(day).zfill(2)}'
        return date + ' ' + time + f':{str(sec).zfill(2)}'
    
    @staticmethod
    def _read_binfile(path: str) -> bytes:
        with open(path, "rb") as bin_file:
            binfile = bin_file.read()
        return binfile
    
    @staticmethod
    def _read_descfile(path: str) -> dict:
        ''' 
        Input: path DESCFILE.INI
        Returns: dict 
            key is index [i]
            value is dict with parameters from .ini
        '''
        config = configparser.ConfigParser()
        config.read(path)
        data_dict = {}
        for section in config.sections():
            section_dict = dict(config.items(section))
            data_dict[int(section)] = section_dict
        return data_dict
    
    def _make_dataframe(self) -> None:
        '''
        Builds data DF, status DF and, if datatype=ex, dates DF.
        '''
        byterows = [self._binfile[i*self._bpr + THIESData.OFFSET : (i+1)*self._bpr ] for i in range(0, self.nrows)]
        data_arr = np.zeros((self.nrows, self.nparameters))
        status_arr = np.zeros((self.nrows, self.nparameters))
        timestamp_arr = np.empty(self.nrows, dtype=object)
        dates_arr = np.empty((self.nrows, self.nparameters), dtype=object)
        
        for i, row in enumerate(byterows):
            # Timestamp
            ts_bytes = self._binfile[i*self._bpr :i*self._bpr + 4]
            ts = THIESData._bytes2datetime(ts_bytes)
            timestamp_arr[i] = ts

            for j in range(self.nparameters):
                # Status = byte 1
                status = row[j*self._bpp]
                status_arr[i, j] = status

                # Value = bytes 2-5, float
                value = struct.unpack('<f', row[j*self._bpp+1 : j*self._bpp+5])[0]
                data_arr[i, j] = round(value,1)

                if self._datatype == 'ex':
                    # Datetime = bytes 6-9
                    datetime = THIESData._bytes2datetime(row[j*self._bpp + 5 : j*self._bpp + 9], only_time=True) 
                    dates_arr[i, j] = datetime
    
        self.dataDF = pd.DataFrame(data_arr).rename(columns={i: self.descfile[i+1]['name'] for i in range(self.nparameters)})
        self.statusDF = pd.DataFrame(status_arr).rename(columns={i: self.descfile[i+1]['name'] for i in range(self.nparameters)})
        self.dataDF = self.dataDF.where(self.statusDF == 0.0, other=None)
        
        if self._datatype == 'ex':
            self.datesDF = pd.DataFrame(dates_arr).rename(columns={i: self.descfile[i+1]['name'] for i in range(self.nparameters)})
            self.datesDF = self.datesDF.where(self.statusDF == 0.0, other=None)
            self.datesDF.index = timestamp_arr
        
        # Add timestamps as index
        self.dataDF.index = timestamp_arr
        self.statusDF.index = timestamp_arr

    
    def write_csv(self, path: str) -> None:
        with open(path, 'w') as outfile:
            outfile.write(self.dataDF.to_csv())
    
    def __repr__(self) -> str:
        return str(self.dataDF)
    
    def _repr_html_(self):
        return self.dataDF._repr_html_()


### Example 1: 2024/05/31 AV files

In [174]:
data1 = THIESData(path_bin_av, path_ini_av, 'av')
data1.dataDF
# data1.write_csv('./example1.csv')

,UBat,Pressure,PrTemp,Precipitation,WS,WD,WS std. dev.,WD std. dev.,Humidity,AirTemperature,Radiation,PicoMoisture 1,PicoSoilTemp 1,PicoMoisture 2,PicoSoilTemp 2,CO2,UVA Radiation,UVB Radiation,PAR Radiation
2024/05/31 00:00:00,11.7,1013.7,8.0,0,0.8,24.0,0.1,24.3,100,5.0,-0.1,None,None,None,None,515.8,-0.1,0,-0.8
2024/05/31 00:10:00,11.7,1013.5,8.0,0,0.6,329.0,0.2,82.1,100,5.0,-0.2,None,None,None,None,506.4,-0.1,0,-0.8
2024/05/31 00:20:00,11.7,1013.4,8.0,0,0.6,12.3,0.2,27.6,100,4.8,-0.2,None,None,None,None,518.1,-0.1,0,-0.7
2024/05/31 00:30:00,11.7,1013.2,7.9,0,0.8,24.6,0.3,25.1,100,4.7,-0.2,None,None,None,None,515.2,-0.1,0,-0.7
2024/05/31 00:40:00,11.7,1013.1,7.8,0,0.8,17.5,0.1,45.2,100,4.7,-0.2,None,None,None,None,504.1,-0.1,0,-0.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024/05/31 23:10:00,11.6,1009.7,5.4,0,0.7,13.8,0.3,44.2,100,3.0,-0.6,None,None,None,None,453.8,-0.1,0,-0.7
2024/05/31 23:20:00,11.6,1009.9,5.4,0,0.5,69.3,0.3,35.9,100,3.0,-0.2,None,None,None,None,451.1,-0.1,0,-0.8
2024/05/31 23:30:00,11.6,1009.8,5.4,0,0.8,69.6,0.4,34.1,100,3.2,-0.3,None,None,None,None,453.7,-0.1,0,-0.7
2024/05/31 23:40:00,11.6,1009.9,5.5,0,1.0,55.2,0.4,51.5,100,3.2,-0.2,None,None,None,None,451.1,-0.1,0,-0.7


### Example 2: 2024/05/31 [EX files](https://www.youtube.com/watch?v=v4-GcS1UQyg)

In [171]:
data2 = THIESData(path_bin_ex, path_ini_ex, 'ex')
data2.dataDF

,UBat MIN,UBat MAX,Pressure MIN,Pressure MAX,PrTemp MIN,PrTemp MAX,WS MIN,WS MAX gust,WD MIN,WD MAX gust,...,PicoSoilTemp 2 MIN,PicoSoilTemp 2 MAX,CO2 MIN,CO2 MAX,UVA Radiation MIN,UVA Radiation MAX,UVB Radiation MIN,UVB Radiation MAX,PAR Radiation MIN,PAR Radiation MAX
2024/05/31 00:00:00,11.7,11.7,1013.6,1013.8,8.0,8.1,0.4,1.2,327.5,16.6,...,None,None,485.7,527.7,-0.1,-0.1,0,0,-0.8,-0.7
2024/05/31 00:10:00,11.7,11.7,1013.4,1013.7,8.0,8.0,0.0,1.0,0.0,97.5,...,None,None,493.5,519.1,-0.1,-0.1,0,0,-0.8,-0.7
2024/05/31 00:20:00,11.7,11.7,1013.1,1013.6,7.9,8.0,0.3,0.9,17.5,7.5,...,None,None,497.5,533.4,-0.1,-0.1,0,0,-0.8,-0.7
2024/05/31 00:30:00,11.7,11.7,1013.0,1013.3,7.9,8.0,0.3,1.4,347.5,37.5,...,None,None,501.1,526.9,-0.1,-0.1,0,0,-0.8,-0.7
2024/05/31 00:40:00,11.7,11.7,1013.0,1013.2,7.8,7.9,0.4,1.1,355.0,85.8,...,None,None,493.6,515.8,-0.1,-0.1,0,0,-0.8,-0.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024/05/31 23:10:00,11.6,11.6,1009.6,1009.9,5.4,5.5,0.3,1.6,122.5,25.8,...,None,None,443.6,463.5,-0.1,-0.1,0,0,-0.8,-0.7
2024/05/31 23:20:00,11.6,11.6,1009.7,1010.0,5.4,5.4,0.0,1.4,0.0,14.1,...,None,None,443.7,457.2,-0.1,-0.1,0,0,-0.8,-0.7
2024/05/31 23:30:00,11.6,11.6,1009.7,1010.0,5.4,5.5,0.0,1.6,0.0,45.0,...,None,None,449.9,456.6,-0.1,-0.1,0,0,-0.8,-0.7
2024/05/31 23:40:00,11.6,11.6,1009.7,1010.1,5.4,5.5,0.3,2.1,230.0,81.6,...,None,None,445.7,457.2,-0.1,-0.1,0,0,-0.8,-0.7


In [172]:
# At what time was the MAX/MIN measurment taken (hh:mm)
data2.datesDF

,UBat MIN,UBat MAX,Pressure MIN,Pressure MAX,PrTemp MIN,PrTemp MAX,WS MIN,WS MAX gust,WD MIN,WD MAX gust,...,PicoSoilTemp 2 MIN,PicoSoilTemp 2 MAX,CO2 MIN,CO2 MAX,UVA Radiation MIN,UVA Radiation MAX,UVB Radiation MIN,UVB Radiation MAX,PAR Radiation MIN,PAR Radiation MAX
2024/05/31 00:00:00,23:50,23:50,23:56,23:53,23:50,23:50,23:59,23:56,23:59,23:56,...,None,None,23:50,23:54,23:50,23:52,23:50,23:52,23:50,23:59
2024/05/31 00:10:00,00:00,00:00,00:09,00:00,00:00,00:00,00:06,00:08,00:06,00:08,...,None,None,00:05,00:00,00:05,00:00,00:05,00:00,00:05,00:04
2024/05/31 00:20:00,00:10,00:11,00:19,00:10,00:15,00:10,00:14,00:16,00:14,00:16,...,None,None,00:17,00:13,00:18,00:14,00:18,00:14,00:18,00:15
2024/05/31 00:30:00,00:20,00:24,00:25,00:23,00:20,00:20,00:21,00:26,00:21,00:26,...,None,None,00:20,00:23,00:20,00:23,00:20,00:27,00:20,00:23
2024/05/31 00:40:00,00:30,00:30,00:38,00:33,00:31,00:30,00:31,00:34,00:31,00:34,...,None,None,00:38,00:32,00:38,00:31,00:38,00:31,00:38,00:31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024/05/31 23:10:00,23:00,23:00,23:01,23:08,23:00,23:00,23:01,23:06,23:01,23:06,...,None,None,23:09,23:00,23:09,23:00,23:09,23:01,23:08,23:05
2024/05/31 23:20:00,23:10,23:11,23:10,23:13,23:10,23:10,23:17,23:19,23:17,23:19,...,None,None,23:10,23:16,23:17,23:16,23:17,23:10,23:17,23:17
2024/05/31 23:30:00,23:22,23:21,23:25,23:20,23:20,23:22,23:29,23:22,23:29,23:22,...,None,None,23:24,23:27,23:20,23:29,23:20,23:22,23:20,23:29
2024/05/31 23:40:00,23:30,23:31,23:30,23:39,23:30,23:30,23:30,23:36,23:30,23:36,...,None,None,23:37,23:30,23:37,23:33,23:37,23:33,23:38,23:33
